In [ ]:
# Instalação das bibliotecas
%pip install -U langgraph langsmith langchain_anthropic

  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached orjson-3.10.16-cp312-cp312-win_amd64.whl.metadata (42 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
  Using cached anthropic-0.49.0-py3-none-any.whl.metadata (24 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.9.0-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata 

In [ ]:
# Inicialização do ambiente
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import random

# Definição do estado
class GuardiaoState(TypedDict):
    media = Optional[bytes]
    tipo = Optional[str]
    resultado_analise = Optional[dict]
    indice_confiabilidade = Optional[int]
    resposta = Optional[str]

# Função dos nós
def receber_input(state: GuardiaoState) -> GuardiaoState:
    """Recebe o input do usuário."""
    print("📥 Recebendo imagem/vídeo via WhatsApp...")  
    return{
        **state,
        "media": b"<simulated_binary_data>",
        "tipo": "video"
    }

def processar_midia(state: GuardiaoState) -> GuardiaoState:
    """Processa o input recebido."""
    print(f"🧪 Processando {state['tipo']}...")
    # Aqui você chamaria um modelo de deepfake detection (ex: HuggingFace, OpenCV, etc)
    resultado_falso = random.choice([True, False])
    explicacao = "Rosto inconsistente com movimento labial detectado." if resultado_falso else "Nenhuma manipulação aparente."
    return {
        **state,
        "resultado_analise": {
        "deepfake_detectado": resultado_falso,
        "explicacao": explicacao
        }
    }

def gerar_indice(state: GuardiaoState) -> GuardiaoState:
    """Gera um índice de confiabilidade."""
    print("📊 Calculando índice de confiabilidade...")
    confianca = random.randint(10,60) if state["resultado_analise"]["deepfake_detectado"] else random.randint(80, 100)
    indice = random.randint(0, 100)
    return {
        **state,
        "indice_confiabilidade": confianca
    }

def gerar_resposta(state: GuardiaoState) -> GuardiaoState:
    """Gera a resposta para o usuário."""
    print("📤 Gerando resposta para o usuário...")
    resposta = (
        f"⚠️ Possível manipulação detectada.\n"
        f"Índice de confiabilidade: {state['indice_confiabilidade']} / 100\n"
        f"Motivo: {state['resultado_analise']['explicacao']}"
        if state["indice_confiabilidade"] < 75
        else "✅ Mídia considerada confiável."
    )

    return {
        **state,
        "resposta": resposta
    }
    



In [ ]:
# Construção do grafo

workflow = StateGraph(GuardiaoState)

workflow.add_node("receber_input", receber_input)
workflow.add_node("processar_midia", processar_midia)
workflow.add_node("gerar_indice", gerar_indice)
workflow.add_node("gerar_resposta", gerar_resposta)

workflow.set_entry_point("receber_input")


workflow.add_edge("receber_input", "processar_media")
workflow.add_edge("processar_media", "gerar_indice")
workflow.add_edge("gerar_indice", "gerar_resposta")
workflow.add_edge("gerar_resposta", END)

graph = workflow.compile()

# Execução do grafo
estado_inicial = {}
final_state = graph.invoke(estado_inicial)


# Exibição do resultado
print("Resultado final:")
print(final_state["resposta"])

📥 Recebendo imagem/vídeo via WhatsApp...


KeyError: 'tipo'

## Próximos passos

In [ ]:
Card 1: Definição de Escopo
Monitorar exclusivamente mensagens de WhatsApp (via API: Twilio, Z‑API, Gupshup etc.).
Conteúdo alvo: imagens e vídeos.
Sinal de desinformação: deepfakes (rostos trocados, vozes sintetizadas, adulterações).
Output principal: índice de confiabilidade de 0 a 100.

Card 2: Coleta de Dados (Ingestão)
Configurar e autenticar bot/API do WhatsApp.
Receber mídia (imagens/vídeos) em um endpoint HTTP.
Armazenar temporariamente os arquivos num bucket ou diretório local.
Disparar evento para o pipeline de análise.

Card 3: Pré‑processamento de Mídia
Vídeo: extrair frames-chave (e.g. 1 frame a cada X segundos).
Imagem: redimensionar / normalizar resolução.
(Opcional) Extrair áudio se for detectar voz sintética.

Card 4: Detecção de Deepfake
Integrar modelo pré‑treinado (ex.: HuggingFace deepfake-detection, FaceForensics++).
Rodar inferência em cada frame/imagem: obter probabilidade de manipulação.
Agregar resultados por vídeo (e.g. média ou percentil de “suspeita”).

Card 5: Geração do Índice de Confiabilidade
Converter a probabilidade de deepfake num score de 0–100:
Exemplo: índice = 100 – int(probabilidade_deepfake * 100).
Definir thresholds (ex.: < 75 alerta, >= 75 confiável).
Registrar explicação resumida (e.g. “inconsistências nos lábios”).

Card 6: Resposta ao Usuário
Formatar mensagem de retorno via WhatsApp:
“⚠️ Vídeo suspeito. Índice: 22/100. Motivo: inconsistência nos lábios.”
“✅ Mídia confiável. Índice: 92/100.”
(Futuro) Extensão para X/Twitter com alertas semelhantes.

Card 7: Modo Educativo(opcional)
Criar fluxo opcional de quizzes rápidos:
Bot pergunta: “O que você observa neste frame adulterado?”
Feedback imediato com dicas de checagem de deepfake.
Agendar envios periódicos (catalogar por temas: rosto, áudio, metadata).

Card 8: Infraestrutura & Deploy(opcional)
Dockerizar cada componente (ingestão, pré-processamento, inferência, bot).
Orquestrar com Kubernetes / Docker Compose / LangGraph.
Monitorar logs e métricas (latência, taxa de falsos positivos).
Planejar credenciais e segurança (tokens das APIs, rate limits).

Card 9: Cronograma de 2 Semanas

Semana	Principais Entregas
1 (dias 1–4)	MVP local: upload manual + detecção + índice
1 (dias 5–7)	Orquestração com LangGraph + testes end‑to‑end
2 (dias 1–3)	Integração com WhatsApp bot (Twilio/Z‑API)
2 (dias 4–5)	Ajustes de UX do bot e implementação do modo educativo
2 (dias 6–7)	Polimento, documentação e apresentação final